Source: https://data.humdata.org/dataset/satellite-detected-water-extent-over-jawa-tengah-province-indonesia-as-of-27-march-2024

In [1]:
import geopandas as gpd
import pandas as pd
import yaml
from rasterio.crs import CRS
from pathlib import Path

In [2]:
yaml_file = '../events/demak_flood_2024.yml' #'porto_algre.yml'
with open(yaml_file) as f:
    event_dict = yaml.safe_load(f)["event"]
event_dict

{'event_name': 'demak_flood_2024',
 'bounds': [87.95007937, 20.62819353, 92.63540544, 26.6340386],
 'event_date': '2024-03-15',
 'pre_event_window_days': 100,
 'post_event_window_days': 60,
 'rtc_track_numbers': [150, 114],
 'mgrs_tiles': ['45QYE', '45QYF', '45QZE', '45QZF'],
 'dist_hls_confirmed_change_min_days': 30,
 'source_id': 'UNOSAT via humanitarian data exchange',
 'links': ['https://data.humdata.org/dataset/satellite-detected-water-extents-between-4-11-july-2024-in-bangladesh',
  'https://data.humdata.org/dataset/satellite-detected-water-extent-over-jawa-tengah-province-indonesia-as-of-27-march-2024']}

In [5]:
df_flood_0 = gpd.read_file('../external_validation_data/unosat/FL20240326IDN_SHP/S1_20240315_FloodExtent_Demak.shp')
df_flood_0 = df_flood_0.to_crs(4326)
df_flood_0.head()

,Water_Clas,Sensor_ID,Sensor_Dat,Confidence,Field_Vali,Water_Stat,Notes,Area_m2,Area_ha,SenorID_ol,...,StaffID,EventCode,d_Water_Cl,d_Sensor_I,d_Confiden,d_Field_Va,d_Water_St,d_SenorID_,d_StaffID,geometry
0,1,42,2024-03-15,4,0,2,None,3.892312e+08,38923.119603,0,...,219,FL20240326IDN,Flood Water,Sentinel-1,To Be Evaluated,Not yet field validated,No water change / Static Level,None,SS,"MULTIPOLYGON (((110.68327 -7.10235, 110.68346 ..."


In [7]:
df_flood_1 = gpd.read_file('../external_validation_data/unosat/FL20240326IDN_SHP/S1_20240315_WaterExtent_Demak.shp')
df_flood_1 = df_flood_1.to_crs(4326)
df_flood_1.head()

,Water_Clas,Sensor_ID,Sensor_Dat,Confidence,Field_Vali,Water_Stat,Notes,Area_m2,Area_ha,SenorID_ol,...,SHAPE_Leng,SHAPE_Area,d_Water_Cl,d_Sensor_I,d_Confiden,d_Field_Va,d_Water_St,d_SenorID_,d_StaffID,geometry
0,2,42,2024-03-15,4,0,2,None,4.743859e+08,47438.586458,0,...,148.315129,0.038812,Satellite Detected Water / Possible Saturated ...,Sentinel-1,To Be Evaluated,Not yet field validated,No water change / Static Level,None,SS,"MULTIPOLYGON (((110.68327 -7.10235, 110.68346 ..."


In [8]:
# df_flood_coastal = df_flood.cx[89.2:91, 21.5: 24].reset_index(drop=True)
# df_flood_coastal.explore()

In [10]:
df_flood_0.total_bounds

array([110.49054153,  -7.10254596, 111.30352226,  -6.68671509])

In [11]:
n = df_flood_0.shape[0]
df_event_f = gpd.GeoDataFrame({'name': [event_dict['event_name'] for k in range(n)],
                               'date': [pd.to_datetime(event_dict['event_date']) for k in range(n)],
                               'source_id': [f'{event_dict["source_id"]}' for k in range(n)]},
                               geometry=df_flood_0.geometry.tolist(),
                               crs=CRS.from_epsg(4326))
df_event_f.head()

,name,date,source_id,geometry
0,demak_flood_2024,2024-03-15,UNOSAT via humanitarian data exchange,"MULTIPOLYGON (((110.68327 -7.10235, 110.68346 ..."


In [12]:
EXTERNAL_VAL_DATA_DIR = Path('../external_validation_data_db/')
EXTERNAL_VAL_DATA_DIR.mkdir(exist_ok=True, parents=True)

In [13]:
df_event_f.to_parquet(EXTERNAL_VAL_DATA_DIR / f'{event_dict["event_name"]}.parquet')